# PyCity Schools Analysis

- Charter schools performed better than district schools. the top 5 schools were all charter and the bottom 5 were all district schools. is it due to the school size requires it's own analysis 
- Schools with higher spending per student did not neccesarily perform better than schools with lower spending budgets.
- Overall passing percentage might have a direct relationship to the size of the school since smaller and medium sized schools performed better than large sized schools but more analysis is required to consider any other factors that might also cause this.

In [1]:
import pandas as pd

In [2]:
# The path to our CSV files
schoolscsv = "schools_complete.csv"
studentscsv = "students_complete.csv"

# Read our csvfiles data into pandas
schools_df = pd.read_csv(schoolscsv)
students_df = pd.read_csv(studentscsv)

In [3]:
schools_df = schools_df.rename(columns={"name": "School Name"})
students_df = students_df.rename(columns={"school": "School Name"})

In [4]:
school_district = pd.merge(students_df, schools_df, on="School Name", suffixes=("_stu", "_sch"))

In [5]:
budget = schools_df["budget"].sum()
schoolscount = schools_df["School ID"].count()
studentscount = students_df["Student ID"].count()
math_mean = students_df["math_score"].mean()
reading_mean = students_df["reading_score"].mean()

In [6]:
district_math = students_df.loc[students_df["math_score"] >= 70,:]["math_score"].count()
district_reading = students_df.loc[students_df["reading_score"] >= 70,:]["reading_score"].count()

In [7]:
math_passing = (district_math/students_df["Student ID"].count())*100
reading_passing = (district_reading/students_df["Student ID"].count())*100

In [8]:
overall_rate = (math_passing+reading_passing)/2

In [9]:
districtsummary_df = pd.DataFrame({
    "Total Schools": schoolscount,
    "Total Students": studentscount,
    "Total Budget": budget,
    "Average Math Score": math_mean,
    "Average Reading Score": reading_mean,
    "% Passing Math": math_passing,
    "% Passing Reading": reading_passing,
    "Overall Passing Rate": overall_rate
}, index=[0])
districtsummary_df
districtsummary_df["Total Budget"] = districtsummary_df["Total Budget"].map("${:,.2f}".format)
districtsummary_df["Total Students"] = districtsummary_df["Total Students"].map("{:,}".format)

## District Summary

In [10]:
reorg_districtsummary = districtsummary_df[["Total Schools","Total Students","Total Budget","Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","Overall Passing Rate"]]
reorg_districtsummary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,80.393158


In [11]:
schoolgrp = school_district.groupby(["School Name"],as_index=False)

In [12]:
school_mean = pd.merge(schools_df, schoolgrp.mean(), on=['School Name'], suffixes=('_Sch','_scgrp'))

In [13]:
del school_mean["School ID_scgrp"]
del school_mean["School ID_Sch"]
del school_mean["budget_scgrp"]
del school_mean["size_scgrp"]
del school_mean["Student ID"]

In [14]:
school_mean = school_mean.rename(columns={"size_Sch": "Total Students","budget_Sch": "Total Budget","reading_score": "Average Reading Score","math_score": "Average Math Score"})

In [15]:
school_mean["Per Student Budget"] = school_mean["Total Budget"]/school_mean["Total Students"]

In [16]:
school_mean1 = pd.DataFrame(school_mean.set_index('School Name'))

In [17]:
pass_reading_df = students_df.loc[students_df["reading_score"] >= 70,:]
pass_math_df = students_df.loc[students_df["math_score"] >= 70,:]

In [18]:
reading_passed_grp = pass_reading_df.groupby("School Name")
math_passed_grp = pass_math_df.groupby("School Name")

In [19]:
school_mean1["% Passing Math"] = (math_passed_grp["Student ID"].count()/school_mean1["Total Students"])*100
school_mean1["% Passing Reading"] = (reading_passed_grp["Student ID"].count()/school_mean1["Total Students"])*100
school_mean1["% Overall Passing Rate"] = (school_mean1["% Passing Math"] + school_mean1["% Passing Reading"])/2
school_mean1 = school_mean1.rename(columns={"type": "School Type", "Total Budget": "Total School Budget"})

In [20]:
reorg_schoolsummary = school_mean1[["School Type","Total Students","Total School Budget","Per Student Budget","Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing Rate"]]

# School Summary

In [21]:
reset_reorg1 = reorg_schoolsummary.sort_index()
reset_reorg1["Total School Budget"] = reset_reorg1["Total School Budget"].map("${:,.2f}".format)
reset_reorg1["Per Student Budget"] = reset_reorg1["Per Student Budget"].map("${:,.2f}".format)
reset_reorg1.index.name = None
reset_reorg1

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,74.306672
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,73.807983
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,94.379391
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270


In [22]:
reset_reorg2 = reset_reorg1.sort_values("% Overall Passing Rate",ascending = False)

# Top Performing Schools (By Passing Rate)

In [23]:
top5schools = reset_reorg2.head(5)
top5schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679


In [24]:
reset_reorg3 = reset_reorg1.sort_values("% Overall Passing Rate")

# Bottom Performing Schools (By Passing Rate)

In [25]:
bottom5schools = reset_reorg3.head(5)
bottom5schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


In [26]:
avgscores = students_df.groupby(['School Name','grade'],as_index=False)

# Math Scores by Grade

In [27]:
avgmath_grp = avgscores["math_score"].mean()
avgmath_table =avgmath_grp.pivot('School Name','grade')
avgmath_table.index.name = None
avgmath_table[avgmath_table.columns.set_levels(['9th', '10th','11th', '12th'], level=1)]

math_score                                 
grade                        9th       10th       11th       12th
Bailey High School     77.083676  76.996772  77.515588  76.492218
Cabrera High School    83.094697  83.154506  82.765560  83.277487
Figueroa High School   76.403037  76.539974  76.884344  77.151369
Ford High School       77.361345  77.672316  76.918058  76.179963
Griffin High School    82.044010  84.229064  83.842105  83.356164
Hernandez High School  77.438495  77.337408  77.136029  77.186567
Holden High School     83.787402  83.429825  85.000000  82.855422
Huang High School      77.027251  75.908735  76.446602  77.225641
Johnson High School    77.187857  76.691117  77.491653  76.863248
Pena High School       83.625455  83.372000  84.328125  84.121547
Rodriguez High School  76.859966  76.612500  76.395626  77.690748
Shelton High School    83.420755  82.917411  83.383495  83.778976
Thomas High School     83.590022  83.087886  83.498795  83.497041
Wilson High School     83.085578  83.724422  83.195326  83.035794
Wright High School     83.264706  84.010288  83.836782  83.644986

# Reading Score by Grade

In [28]:
avgread_grp = avgscores["reading_score"].mean()
avgread_table =avgread_grp.pivot('School Name','grade')
avgread_table.index.name = None
avgread_table[avgread_table.columns.set_levels(['9th', '10th','11th', '12th'], level=1)]

reading_score                                 
grade                           9th       10th       11th       12th
Bailey High School        81.303155  80.907183  80.945643  80.912451
Cabrera High School       83.676136  84.253219  83.788382  84.287958
Figueroa High School      81.198598  81.408912  80.640339  81.384863
Ford High School          80.632653  81.262712  80.403642  80.662338
Griffin High School       83.369193  83.706897  84.288089  84.013699
Hernandez High School     80.866860  80.660147  81.396140  80.857143
Holden High School        83.677165  83.324561  83.815534  84.698795
Huang High School         81.290284  81.512386  81.417476  80.305983
Johnson High School       81.260714  80.773431  80.616027  81.227564
Pena High School          83.807273  83.612000  84.335938  84.591160
Rodriguez High School     80.993127  80.629808  80.864811  80.376426
Shelton High School       84.122642  83.441964  84.373786  82.781671
Thomas High School        83.728850  84.254157  83.585542  83.831361
Wilson High School        83.939778  84.021452  83.764608  84.317673
Wright High School        83.833333  83.812757  84.156322  84.073171

In [29]:
# Create bins
bins = [575,585,615,645,675]

# Create labels for these bins
group_labels = ["<$585", "\$585-\$615", "\$615-\$645", "\$645-\$675"]

perStudent_series = pd.cut(reorg_schoolsummary["Per Student Budget"], bins, labels=group_labels)

In [30]:
reorg_schoolsummary2 = reorg_schoolsummary
reorg_schoolsummary2["Spending ranges (Per Student)"] = perStudent_series

C:\Users\mywor\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


# Scores by School Spending

In [31]:
perStudent_grp = reorg_schoolsummary2.groupby("Spending ranges (Per Student)")
perStudent_grp["Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing Rate"].mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
\$585-\$615,83.599686,83.885211,94.230858,95.900287,95.065572
\$615-\$645,79.079225,81.891436,75.668212,86.106569,80.887391
\$645-\$675,76.997210,81.027843,66.164813,81.133951,73.649382


In [32]:
# Create bins in which to place values based upon TED Talk views
bins1 = [425,1000,2000,5000]

# Create labels for these bins
group_labels1 = ["Small(<1000)", "Medium(1000-2000)", "Large(2000-5000)"]

schoolsize_series = pd.cut(reorg_schoolsummary["Total Students"], bins1, labels=group_labels1)

In [33]:
reorg_schoolsummary3 = reorg_schoolsummary
reorg_schoolsummary3["School Size"] = schoolsize_series

C:\Users\mywor\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


# Scores by School Size

In [34]:
schoolsize_grp = reorg_schoolsummary3.groupby("School Size")
schoolsize_grp["Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing Rate"].mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small(<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium(1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large(2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


# Scores by School Type

In [35]:
schooltype_grp = reorg_schoolsummary.groupby("School Type")
schooltype_grp["Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing Rate"].mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
